In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import gzip
import csv
f=gzip.open('amazon_reviews_us_Gift_Card_v1_00.tsv.gz','rt') 
reader=csv.reader(f,delimiter='\t');header=next(reader)
dataset=[]
for line in reader:
  d=dict(zip(header,line))
  for field in['helpful_votes','star_rating','total_votes']:
    d[field]=int(d[field])
  dataset.append(d)

In [2]:
from collections import defaultdict
wordcount=defaultdict(int)
for d in dataset:
  for w in d['review_body'].split():
    wordcount[w]+=1
print(len(wordcount))

96967


In [3]:
wordcount=defaultdict(int)
import string
for d in dataset:
  r="".join([c for c in d['review_body'].lower() if c not in string.punctuation])
  for w in r.split():
    wordcount[w]+=1
print(len(wordcount))

46142


In [4]:
counts=[(wordcount[w],w) for w in wordcount]
counts.sort()
counts.reverse()
words=[x[1] for x in counts[:1000]]
wordid=dict(zip(words,range(len(words))))
wordset=set(words)
print(len(wordset))

1000


In [5]:
import string
def feature(datum):
  feat=[0]*len(words)
  r=''.join([c for c in datum['review_body'].lower() if not c in string.punctuation])
  for w in r.split():
    if w in words:
      feat[wordid[w]]+=1
  feat.append(1)
  return feat

In [6]:
import random
import numpy
random.shuffle(dataset)
x=[feature(d) for d in dataset]
y=[d['star_rating'] for d in dataset]
y[-10:]

[5, 5, 5, 5, 2, 5, 5, 5, 5, 5]

In [7]:
N=len(x)
x_train=x[:N//2]
x_valid=x[N//2:3*N//4]
x_test=x[3*N//4:]
y_train=y[:N//2]
y_valid=y[N//2:3*N//4]
y_test=y[3*N//4:]

In [ ]:
print(len(x))
print(len(x_train))
print(len(x_valid))
print(len(x_test))

148310
74155
37077
37078


In [ ]:
def MSE(model,x,y):
  predictions=model.predict(x)
  differences=[(a-b)**2 for (a,b) in zip(predictions,y)]
  return sum(differences)/len(differences)

In [ ]:
bestModel=None
bestMSE=None
from sklearn import linear_model
for lamb in [0.01,0.1,1,10,100]:
  model=linear_model.Ridge(lamb,fit_intercept=False)
  model.fit(x_train,y_train)
  mseTrain=MSE(model,x_train,y_train)
  msevalid=MSE(model,x_valid,y_valid)
  mseTrain=MSE(model,x_train,y_train)
  mseTrain=MSE(model,x_train,y_train)
  print("lambda="+str(lamb)+",training/validation error="+str(mseTrain)+'\n'+str(msevalid))
  if not bestModel or msevalid<bestMSE:
    bestModel=model
    bestMSE=msevalid

Ridge(alpha=0.01, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

lambda=0.01,training/validation error=0.4213996739143779
0.4700091585933161


Ridge(alpha=0.1, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

lambda=0.1,training/validation error=0.4213996828376349
0.46998010685012703


Ridge(alpha=1, copy_X=True, fit_intercept=False, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

lambda=1,training/validation error=0.42140056517488805
0.46969423844462815


Ridge(alpha=10, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

lambda=10,training/validation error=0.42147981378702903
0.46724869605282465


Ridge(alpha=100, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

lambda=100,training/validation error=0.4251828458370377
0.4605713255179781


In [ ]:
print(bestModel)
print(bestMSE)

Ridge(alpha=100, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
0.4605713255179781


In [ ]:
mseTest=MSE(bestModel,x_test,y_test)
print("testerror"+str(mseTest))

testerror0.4312712101314393
